# Analysis of the validation times and validation attempts on the test set

In [10]:
import pandas as pd

merged_full_pipeline_df = pd.read_csv("merged_full_pipeline.csv")
summary_full_pipeline_df = pd.read_csv("summary_full_pipeline.csv")
merged_baseline_pipeline_df = pd.read_csv("merged_baseline_pipeline.csv")
summary_baseline = pd.read_csv("summary_baseline.csv")
merged_sparql_llm_pipeline_df = pd.read_csv("merged_sparql_llm_pipeline.csv")
summary_sparql_llm_pipeline_df = pd.read_csv("summary_sparql_llm_pipeline.csv")

### Only a single nonempty validation time in the other two test sets

In [13]:
import ast

# Check for nonempty lists in 'validation_time' for merged_full_pipeline_df
nonempty_validation_full = merged_full_pipeline_df['validation_time'].apply(lambda x: isinstance(x, list) and len(x) > 0).any()

# Check for nonempty lists in 'validation_time' for merged_sparql_llm_pipeline_df
nonempty_validation_sparql_llm = merged_sparql_llm_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0).any()
# Count nonempty lists in 'validation_time' for merged_full_pipeline_df
count_nonempty_full = merged_full_pipeline_df['validation_time'].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()

# Count nonempty lists in 'validation_time' for merged_sparql_llm_pipeline_df
count_nonempty_sparql_llm = merged_sparql_llm_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0).sum()

print("Count of nonempty lists in merged_full_pipeline_df:", count_nonempty_full)
print("Count of nonempty lists in merged_sparql_llm_pipeline_df:", count_nonempty_sparql_llm)
print("Nonempty lists in merged_full_pipeline_df:", nonempty_validation_full)
print("Nonempty lists in merged_sparql_llm_pipeline_df:", nonempty_validation_sparql_llm)

Count of nonempty lists in merged_full_pipeline_df: 0
Count of nonempty lists in merged_sparql_llm_pipeline_df: 1
Nonempty lists in merged_full_pipeline_df: False
Nonempty lists in merged_sparql_llm_pipeline_df: True


### More in baseline


In [14]:

valid_nonempty_validation_df = merged_baseline_pipeline_df[
    (merged_baseline_pipeline_df['valid_query']) &
    (merged_baseline_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0))
]

invalid_nonempty_validation_df = merged_baseline_pipeline_df[
    (~merged_baseline_pipeline_df['valid_query']) &
    (merged_baseline_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0))
]

In [15]:
valid_nonempty_validation_df['attempts'].value_counts()

attempts
1    31
2     7
3     2
Name: count, dtype: int64

Total attempts fully failed:

In [16]:

invalid_nonempty_validation_df['attempts'].value_counts()

attempts
3    2
Name: count, dtype: int64